In [1]:
import pandas as pd
import mysql.connector

# MySQL 연결 정보
db_config = {
    "host": "jkkyui-MacBookPro.local",         # MySQL 서버 호스트
    "user": "root",     # MySQL 사용자 이름
    "password": "apdlvmf4@@", # MySQL 비밀번호
    "database": "Semose_DB"  # 데이터베이스 이름
}

# MySQL 서버에 연결
connection = mysql.connector.connect(**db_config)

# 테이블 이름
table_name = "places"

# SQL 쿼리를 사용하여 데이터를 데이터프레임으로 가져옵니다.
query = f"SELECT * FROM {table_name}"
df = pd.read_sql(query, connection)

# MySQL 연결 닫기
connection.close()

# 데이터프레임 출력
print(df)

              id           address_name category_group_code  \
0     1000979638  경기 수원시 팔달구 인계동 241-14                None   
1     1001888126      경기 수원시 장안구 송죽동 79                 CE7   
2       10018908  경기 수원시 영통구 매탄동 1264-2                 FD6   
3      100245369   경기 수원시 장안구 영화동 163-2                 FD6   
4       10028799  경기 수원시 영통구 매탄동 153-22                 HP8   
...          ...                    ...                 ...   
5416     9979794   경기 수원시 팔달구 우만동 134-4                 PM9   
5417   998502881     경기 수원시 영통구 하동 1016                 HP8   
5418     9985403     경기 수원시 팔달구 인계동 368                 PM9   
5419     9988730   경기 수원시 팔달구 인계동 955-8                 FD6   
5420   999313761    경기 수원시 팔달구 남창동 68-5                 BK9   

     category_group_name              category_name distance          phone  \
0                   None                가정,생활 > 세탁소     None   031-215-6030   
1                     카페  음식점 > 카페 > 커피전문점 > 커피에반하다     None  031-8067-7005   
2     

In [2]:
df.x

0       127.027444
1       127.004149
2       127.044925
3       127.015670
4       127.043829
           ...    
5416    127.038035
5417    127.060685
5418    127.036174
5419    127.028941
5420    127.014591
Name: x, Length: 5421, dtype: float64

In [3]:
df.y

0       37.276103
1       37.306363
2       37.259857
3       37.291204
4       37.274432
          ...    
5416    37.278608
5417    37.288157
5418    37.271649
5419    37.274731
5420    37.280977
Name: y, Length: 5421, dtype: float64

In [4]:
import pandas as pd
import folium
from folium import plugins

# 중심 위치 설정
center_latitude = df.y.mean()
center_longitude = df.x.mean()

# Folium 지도 생성
m = folium.Map(location=[center_latitude, center_longitude], zoom_start=5)

# DataFrame의 각 행에 대한 마커 생성
for index, row in df[['y','x']].iterrows():
    folium.CircleMarker([row['y'], row['x']], radius=5, color='blue').add_to(m)

# 지도를 HTML로 저장
m.save('/Users/jkky/Downloads/map.html')

In [5]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # 지구의 반지름 (킬로미터)
    radius = 6371

    # 위도 및 경도를 라디안으로 변환
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Haversine 공식을 사용하여 두 지점 간의 거리 계산
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = radius * c

    return distance

In [6]:
def change_min(lat1, lon1, lat2, lon2): # 도보 걸음걸이 속도 = 67m/min
    distance = haversine(lat1, lon1, lat2, lon2)
    min_ = distance * 1000 / 67
    return min_

In [7]:
from change import get_lat_lng_from_address

lat, lon = get_lat_lng_from_address('팔달구 인계동 374-7')

In [8]:
# DataFrame에 있는 위도와 경도 칼럼을 사용하여 거리 계산
df['거리(분)'] = df.apply(lambda row: change_min(lat, lon, row['y'], row['x']), axis=1)

# DataFrame 출력
print(df)

              id           address_name category_group_code  \
0     1000979638  경기 수원시 팔달구 인계동 241-14                None   
1     1001888126      경기 수원시 장안구 송죽동 79                 CE7   
2       10018908  경기 수원시 영통구 매탄동 1264-2                 FD6   
3      100245369   경기 수원시 장안구 영화동 163-2                 FD6   
4       10028799  경기 수원시 영통구 매탄동 153-22                 HP8   
...          ...                    ...                 ...   
5416     9979794   경기 수원시 팔달구 우만동 134-4                 PM9   
5417   998502881     경기 수원시 영통구 하동 1016                 HP8   
5418     9985403     경기 수원시 팔달구 인계동 368                 PM9   
5419     9988730   경기 수원시 팔달구 인계동 955-8                 FD6   
5420   999313761    경기 수원시 팔달구 남창동 68-5                 BK9   

     category_group_name              category_name distance          phone  \
0                   None                가정,생활 > 세탁소     None   031-215-6030   
1                     카페  음식점 > 카페 > 커피전문점 > 커피에반하다     None  031-8067-7005   
2     

In [9]:
df[df['거리(분)'] < 10]

,id,address_name,category_group_code,category_group_name,category_name,distance,phone,place_name,place_url,road_address_name,x,y,거리(분)
4,10028799,경기 수원시 영통구 매탄동 153-22,HP8,병원,"의료,건강 > 병원 > 치과",None,031-8013-7575,환한미소 바른이치과의원,http://place.map.kakao.com/10028799,경기 수원시 영통구 중부대로256번길 3,127.043829,37.274432,9.258853
25,10110241,경기 수원시 영통구 매탄동 176-2,None,None,"가정,생활 > 세탁소",None,None,현대세탁,http://place.map.kakao.com/10110241,경기 수원시 영통구 권광로260번길 36,127.041960,37.272317,6.205614
26,1011328109,경기 수원시 영통구 매탄동 153-15,None,None,"가정,생활 > 여가시설 > 노래방 > 코인노래방",None,None,아지트코인노래연습장,http://place.map.kakao.com/1011328109,경기 수원시 영통구 중부대로 260,127.044162,37.274346,9.624761
33,10137097,경기 수원시 팔달구 우만동 92-1,BK9,은행,"금융,보험 > 금융서비스 > 은행 > KB국민은행",None,031-216-8233,KB국민은행 우만동지점,http://place.map.kakao.com/10137097,경기 수원시 팔달구 중부대로223번길 4,127.040821,37.276005,7.512378
56,10203291,경기 수원시 팔달구 인계동 943,FD6,음식점,음식점 > 분식,None,031-223-0187,김밥하우스,http://place.map.kakao.com/10203291,경기 수원시 팔달구 장다리로 304,127.030691,37.274963,9.616371
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5345,9769963,경기 수원시 팔달구 우만동 141-4,None,None,"의료,건강 > 한약국,한약방",None,None,동양한약방,http://place.map.kakao.com/9769963,경기 수원시 팔달구 중부대로 187,127.036754,37.277231,7.927290
5354,9787246,경기 수원시 팔달구 인계동 158-58,None,None,"서비스,산업 > 인쇄,복사",None,031-211-1992,동남문화사,http://place.map.kakao.com/9787246,경기 수원시 팔달구 권광로312번길 5,127.037565,37.275058,4.310347
5397,9919486,경기 수원시 팔달구 인계동 158-8,FD6,음식점,음식점 > 한식 > 감자탕,None,031-211-0456,1인자 감자탕뼈해장국,http://place.map.kakao.com/9919486,경기 수원시 팔달구 중부대로206번길 4,127.038212,37.276170,6.263699
5402,9942293,경기 수원시 팔달구 우만동 95-1,FD6,음식점,음식점 > 한식 > 국수 > 칼국수,None,031-211-6886,이모네손칼국수,http://place.map.kakao.com/9942293,경기 수원시 팔달구 중부대로223번길 20-7,127.041313,37.276880,9.060869


In [14]:
# 만약
# 클라이언트가 스타벅스, 맥도날드, 올리브영, 크린토피아를 선택했을 시에

query = f"SELECT * FROM {table_name} WHERE place_name LIKE '%스타벅스%' OR place_name LIKE '%크린토피아%'"


# MySQL 서버에 연결
connection = mysql.connector.connect(**db_config)

# 테이블 이름
table_name = "places"

# SQL 쿼리를 사용하여 데이터를 데이터프레임으로 가져옵니다.
query = f"SELECT * FROM {table_name} WHERE place_name LIKE '%스타벅스%' OR place_name LIKE '%크린토피아%'"
df = pd.read_sql(query, connection)

# MySQL 연결 닫기
connection.close()

# 데이터프레임 출력
print(df)

            id            address_name category_group_code  \
0   1110332114      경기 수원시 팔달구 신풍동 224                 CE7   
1   1114866709    경기 수원시 장안구 영화동 403-3                None   
2     11316780  경기 수원시 팔달구 매산로1가 53-11                 CE7   
3   1140931343     경기 수원시 장안구 정자동 79-7                 CE7   
4   1145732557     경기 수원시 영통구 이의동 1257                 CE7   
..         ...                     ...                 ...   
74   827807269   경기 수원시 영통구 매탄동 153-20                None   
75   836826800    경기 용인시 기흥구 영덕동 974-3                None   
76   934570533    경기 수원시 권선구 세류동 325-1                 CE7   
77   937825753      경기 수원시 영통구 원천동 589                None   
78   991210048    경기 수원시 팔달구 우만동 153-7                 PK6   

   category_group_name                     category_name distance  \
0                   카페           음식점 > 카페 > 커피전문점 > 스타벅스     None   
1                 None  가정,생활 > 세탁소 > 셀프빨래방 > 크린토피아 코인워시     None   
2                   카페           음식점 > 카페 > 커피전문